# 问题七：股票交易最大利润（带手续费和冷却期）

## 题目背景

你正在为一个高频交易系统编写回测引擎。给定一个整数数组 `prices`，其中 `prices[i]` 表示某支股票在第 `i` 天的价格。

你需要设计一个算法来计算最大利润，并严格遵守交易所的结算制度与佣金模型。

## 题目描述

请计算在满足以下所有规则的情况下，你能获得的最大利润：

1. **无限次交易**：你可以尽可能多地进行买卖交易（买入和卖出），但你必须在再次买入前卖出掉之前的股票（即手中最多只能持有一股）。

2. **交易成本**：每次**卖出**股票时，需要支付一笔固定的手续费 `fee`。

3. **结算延迟（冷却期）**：卖出股票后，需要等待 `D` 天的资金结算期才能再次买入。
   - 例如：如果你在第 `i` 天卖出股票，那么你在第 `i+1, ..., i+D` 天都无法买入股票。
   - 你最早可以在第 `i + D + 1` 天再次买入。

## 约束条件

- `1 ≤ prices.length ≤ 5 × 10⁴`
- `0 ≤ prices[i] ≤ 5 × 10⁴`
- `0 ≤ D ≤ prices.length`
- `0 ≤ fee ≤ 5 × 10⁴`

## 示例

### Example 1:
```
Input: prices = [1, 2, 3, 0, 2], D = 1, fee = 0
Output: 3
Explanation:
- 第0天买入 (price 1)，第1天卖出 (price 2)，利润 = 2 - 1 - 0 = 1
- 第2天处于冷冻期 (D=1)，无法操作
- 第3天买入 (price 0)，第4天卖出 (price 2)，利润 = 2 - 0 - 0 = 2
- 总利润 = 1 + 2 = 3
```

### Example 2:
```
Input: prices = [1, 5, 2, 4, 3], D = 2, fee = 1
Output: 3
Explanation:
- 第0天买入 (price 1)，第1天卖出 (price 5)
- 支付 fee = 1，本次净赚 5 - 1 - 1 = 3
- 卖出后进入2天冷却（第2、3天禁买）
- 之后无更优操作，总利润 3
```

---

## 💡 解题思路提示

这是一个经典的动态规划问题。你可以考虑用状态机的思想来建模：

### 状态定义

对于第 `i` 天，我们可以定义以下状态：

1. **`hold[i]`**: 第 `i` 天持有股票时的最大利润
2. **`free[i]`**: 第 `i` 天不持有股票且可以买入时的最大利润
3. **`cooldown[i][k]`**: 第 `i` 天处于冷却期第 `k` 天时的最大利润 (k = 1, 2, ..., D)

### 状态转移

思考一下：
- 如何从前一天的状态转移到当前状态？
- 买入、卖出、持有、冷却各自对应什么状态转移？
- 手续费在哪个时刻扣除？

---

## 📝 请在下方实现你的解决方案

**注意**：请只修改函数内部的实现，不要修改函数签名。

In [1]:
def maxProfit(prices, D, fee):
    n = len(prices)
    if n == 0:
        return 0

    # hold[i]: 第i天持有股票的最大收益
    # cool[i]: 第i天不持有股票的最大收益
    hold = [-float('inf')] * n
    cool = [0] * n

    hold[0] = -prices[0]

    for i in range(1, n):
        # 今天持有 = max(昨天持有, 从冷冻期结束后买入)
        if i > D:
            hold[i] = max(hold[i-1], cool[i-D-1] - prices[i])
        else:
            # 冷冻期内只能从初始状态买入
            hold[i] = max(hold[i-1], -prices[i])

        # 今天不持有 = max(昨天不持有, 今天卖出)
        cool[i] = max(cool[i-1], hold[i-1] + prices[i] - fee)

    return max(0, cool[n-1])

---

## 🧪 测试案例

运行以下单元格来测试你的实现。

In [2]:
# 测试案例
def run_tests():
    test_cases = [
        # (prices, D, fee, expected_output, description)
        ([1, 2, 3, 0, 2], 1, 0, 3, "示例1：基本冷却期测试"),
        ([1, 5, 2, 4, 3], 2, 1, 3, "示例2：较长冷却期 + 手续费"),
        ([1, 3, 2, 8, 4, 9], 1, 2, 6, "测试3：多次交易"),
        ([1, 2, 3, 4, 5], 0, 0, 4, "测试4：单调递增，无冷却期"),
        ([5, 4, 3, 2, 1], 1, 0, 0, "测试5：单调递减，不交易"),
        ([1, 3, 7, 5, 10, 3], 2, 3, 6, "测试6：高手续费"),
        ([1], 1, 0, 0, "测试7：单个价格"),
        ([1, 2], 0, 0, 1, "测试8：两个价格，无冷却"),
        ([1, 4, 2], 1, 0, 3, "测试9：买卖一次最优"),
        ([2, 1, 4, 5, 2, 9, 7], 0, 1, 9, "测试10：无冷却期，有手续费"),
    ]
    
    passed = 0
    failed = 0
    
    print("="*80)
    print("开始测试...")
    print("="*80)
    
    for i, (prices, D, fee, expected, description) in enumerate(test_cases, 1):
        try:
            result = maxProfit(prices, D, fee)
            status = "✅ PASS" if result == expected else "❌ FAIL"
            
            if result == expected:
                passed += 1
            else:
                failed += 1
            
            print(f"\n测试 {i}: {description}")
            print(f"  输入: prices={prices}, D={D}, fee={fee}")
            print(f"  期望输出: {expected}")
            print(f"  实际输出: {result}")
            print(f"  状态: {status}")
            
        except Exception as e:
            failed += 1
            print(f"\n测试 {i}: {description}")
            print(f"  输入: prices={prices}, D={D}, fee={fee}")
            print(f"  ❌ 运行出错: {str(e)}")
    
    print("\n" + "="*80)
    print(f"测试完成！通过: {passed}/{len(test_cases)}, 失败: {failed}/{len(test_cases)}")
    print("="*80)
    
    if passed == len(test_cases):
        print("\n🎉 恭喜！所有测试通过！")
    else:
        print(f"\n⚠️  还有 {failed} 个测试未通过，请继续努力！")

# 运行测试
run_tests()

开始测试...

测试 1: 示例1：基本冷却期测试
  输入: prices=[1, 2, 3, 0, 2], D=1, fee=0
  期望输出: 3
  实际输出: 3
  状态: ✅ PASS

测试 2: 示例2：较长冷却期 + 手续费
  输入: prices=[1, 5, 2, 4, 3], D=2, fee=1
  期望输出: 3
  实际输出: 3
  状态: ✅ PASS

测试 3: 测试3：多次交易
  输入: prices=[1, 3, 2, 8, 4, 9], D=1, fee=2
  期望输出: 6
  实际输出: 6
  状态: ✅ PASS

测试 4: 测试4：单调递增，无冷却期
  输入: prices=[1, 2, 3, 4, 5], D=0, fee=0
  期望输出: 4
  实际输出: 4
  状态: ✅ PASS

测试 5: 测试5：单调递减，不交易
  输入: prices=[5, 4, 3, 2, 1], D=1, fee=0
  期望输出: 0
  实际输出: 0
  状态: ✅ PASS

测试 6: 测试6：高手续费
  输入: prices=[1, 3, 7, 5, 10, 3], D=2, fee=3
  期望输出: 6
  实际输出: 6
  状态: ✅ PASS

测试 7: 测试7：单个价格
  输入: prices=[1], D=1, fee=0
  期望输出: 0
  实际输出: 0
  状态: ✅ PASS

测试 8: 测试8：两个价格，无冷却
  输入: prices=[1, 2], D=0, fee=0
  期望输出: 1
  实际输出: 1
  状态: ✅ PASS

测试 9: 测试9：买卖一次最优
  输入: prices=[1, 4, 2], D=1, fee=0
  期望输出: 3
  实际输出: 3
  状态: ✅ PASS

测试 10: 测试10：无冷却期，有手续费
  输入: prices=[2, 1, 4, 5, 2, 9, 7], D=0, fee=1
  期望输出: 9
  实际输出: 9
  状态: ✅ PASS

测试完成！通过: 10/10, 失败: 0/10

🎉 恭喜！所有测试通过！
